Rede neural CNN treinado sob dados brutos de forma binária

Importar as bibliotecas e os dados do conjunto MNIST

In [48]:
import json
import numpy as np
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist

# 1. Carregar o conjunto de dados MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
#print (x_train[0][1]) # [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
#print(x_test)

(60000, 28, 28)
(10000, 28, 28)
(60000,)


Normalizar e redimensionar os dados

In [45]:
x_train = x_train.reshape((60000, 28, 28, 1)).astype('float32') / 255
x_test = x_test.reshape((10000, 28, 28, 1)).astype('float32') / 255
print(x_train.shape)

(60000, 28, 28, 1)


Criar os filtros de classes

In [35]:
# Filtrar apenas duas classes (por exemplo, 0 e 1)
binary_classes = [0, 1]
train_filter = np.where((y_train == binary_classes[0]) | (y_train == binary_classes[1]))
test_filter = np.where((y_test == binary_classes[0]) | (y_test == binary_classes[1]))

Filtrar as classes

In [36]:
x_train, y_train = x_train[train_filter], y_train[train_filter]
x_test, y_test = x_test[test_filter], y_test[test_filter]

**Construir o modelo da CNN**

Este modelo é uma rede neural convolucional (CNN) construída usando a API Sequential do Keras. Ele é projetado para processar imagens de entrada com a forma (28, 28, 1), o que sugere imagens em escala de cinza de 28x28 pixels. Aqui está uma explicação passo a passo de cada camada:

Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)): Esta é a primeira camada convolucional. Ela tem 32 filtros (ou kernels), cada um com um tamanho de (3, 3). A função de ativação ReLU é usada. input_shape=(28, 28, 1) define a forma das imagens de entrada que a rede espera, que são imagens de 28x28 pixels em escala de cinza (1 canal de cor).

MaxPooling2D((2, 2)): Uma camada de pooling que reduz as dimensões espaciais (altura e largura) da saída da camada anterior pela metade. Isso é feito usando uma janela de pooling de tamanho (2, 2) e selecionando o valor máximo dentro dessa janela.

Conv2D(64, (3, 3), activation='relu'): Outra camada convolucional com 64 filtros de tamanho (3, 3) e função de ativação ReLU. Não é necessário especificar o formato de entrada aqui, pois o Keras infere automaticamente a partir da saída da camada anterior.

MaxPooling2D((2, 2)): Mais uma camada de pooling que reduz as dimensões espaciais da saída da camada anterior pela metade, usando a mesma janela de pooling de tamanho (2, 2).

Conv2D(64, (3, 3), activation='relu'): Uma terceira camada convolucional com 64 filtros de tamanho (3, 3) e função de ativação ReLU. Isso aumenta ainda mais a capacidade da rede de aprender características complexas.

Flatten(): Esta camada achata a saída tridimensional da última camada convolucional em um vetor unidimensional. Isso é necessário porque as próximas camadas, as camadas densas, esperam entradas unidimensionais.

Dense(64, activation='relu'): Uma camada densa (ou totalmente conectada) com 64 unidades e função de ativação ReLU. Ela recebe o vetor achatado da camada anterior como entrada.

Dense(1, activation='sigmoid'): A última camada é uma camada densa com uma única unidade e a função de ativação sigmoid. Isso é típico para problemas de classificação binária, onde a saída é a probabilidade de a entrada pertencer a uma das duas classes.

Este modelo é adequado para tarefas de classificação binária de imagens, como determinar se uma imagem contém um certo objeto ou não.

In [37]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

Compilar o modelo - https://www.tensorflow.org/api_docs/python/tf/keras/Model#compile

Utiliza o otimizador "adam" - https://www.tensorflow.org/api_docs/python/tf/keras/optimizers

Este código configura o modelo para o treinamento, especificando o otimizador, a função de perda e as métricas a serem avaliadas durante o treinamento e a validação:

optimizer='adam': Define o otimizador Adam para ajustar os pesos da rede. Adam é um algoritmo de otimização baseado em gradiente que é eficiente em termos de computação e tem um requisito de memória relativamente baixo. É amplamente usado por ser eficaz em uma ampla gama de problemas de aprendizado de máquina.

loss='binary_crossentropy': Especifica a função de perda como entropia cruzada binária. Esta é uma escolha comum para problemas de classificação binária. A entropia cruzada binária mede o desempenho do modelo cuja saída é um valor de probabilidade entre 0 e 1. Ela compara a distribuição de probabilidade prevista pela rede com a distribuição real (os rótulos verdadeiros).

metrics=['accuracy']: Define a métrica de avaliação do modelo como precisão (accuracy). A precisão é a fração de previsões corretas entre o total de previsões feitas. É uma métrica comum para avaliar o desempenho de modelos em tarefas de classificação.

Ao chamar model.compile(), você está preparando o modelo para o treinamento, compilando-o com as configurações especificadas. Isso inclui preparar as estruturas de dados internas do modelo e se preparar para a otimização (ajuste dos pesos da rede durante o treinamento).

In [38]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

- Criando e salvando os hiperparâmetros da arquitetura e da inicialização em formato json em arquivos-brutos/hiperparametros.json

- Salvando os pesos iniciais

In [39]:
# Definir os hiperparâmetros da arquitetura da rede neural e os hiperparâmetros de inicialização
hiperparametros = {
    'arquitetura': {
        'camadas': [
            {'tipo': 'Conv2D', 'filtros': 32, 'kernel_size': (3, 3), 'activation': 'relu', 'input_shape': (28, 28, 1)},
            {'tipo': 'MaxPooling2D', 'pool_size': (2, 2)},
            {'tipo': 'Conv2D', 'filtros': 64, 'kernel_size': (3, 3), 'activation': 'relu'},
            {'tipo': 'MaxPooling2D', 'pool_size': (2, 2)},
            {'tipo': 'Conv2D', 'filtros': 64, 'kernel_size': (3, 3), 'activation': 'relu'},
            {'tipo': 'Flatten'},
            {'tipo': 'Dense', 'unidades': 64, 'activation': 'relu'},
            {'tipo': 'Dense', 'unidades': 1, 'activation': 'sigmoid'}
        ]
    },
    'inicializacao': {
        'optimizer': 'adam',
        'loss': 'binary_crossentropy',
        'metrics': ['accuracy']
    }
}

# Serializando os hiperparâmetros em uma string JSON
hiperparametros_json = json.dumps(hiperparametros, indent=4)

# Escrevendo a string JSON em um arquivo
with open("arquivos-brutos/hiperparametros.json", "w") as arquivo:
    arquivo.write(hiperparametros_json)


# PESOS INICIAIS
model.save_weights('arquivos-brutos/pesos_iniciais.weights.h5')

**Treinar o modelo**

Este trecho de código inicia o treinamento do modelo com os dados fornecidos, especificando também como o treinamento deve ser realizado. Aqui está o que cada argumento significa:

x_train: Os dados de entrada para treinar o modelo. Estes são os exemplos que o modelo usará para aprender.

y_train: As etiquetas (rótulos) correspondentes aos dados de entrada x_train. Estas são as respostas corretas que o modelo tentará prever.

epochs=15: Define o número de épocas de treinamento. Uma época significa uma iteração sobre todos os dados de entrada. Portanto, o modelo passará pelos dados de treinamento 15 vezes.

batch_size=64: Especifica o tamanho do lote (batch size). Isso significa que 64 exemplos de x_train e y_train serão usados para atualizar os pesos do modelo de uma vez. O uso de lotes ajuda a acelerar o treinamento e pode contribuir para uma melhor generalização do modelo.

validation_split=0.2: Este argumento reserva 20% dos dados de treinamento para validação. O modelo não treinará nesses dados. Em vez disso, após cada época, ele avaliará seu desempenho nesse subconjunto de validação. Isso é útil para monitorar o overfitting (quando o modelo aprende padrões específicos dos dados de treinamento, mas falha em generalizar para novos dados).

Portanto, este código treina o modelo nos dados fornecidos (x_train e y_train), por 15 épocas, usando lotes de 64 exemplos, e avalia o desempenho do modelo em 20% dos dados de treinamento reservados para validação após cada época.

In [40]:
historico = model.fit(x_train, y_train, epochs=10, batch_size=10, validation_split=0.2)

Epoch 1/10
1014/1014 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9901 - loss: 0.0418 - val_accuracy: 0.9984 - val_loss: 0.0023
Epoch 2/10
1014/1014 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9983 - loss: 0.0040 - val_accuracy: 0.9996 - val_loss: 7.5968e-04
Epoch 3/10
1014/1014 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9993 - loss: 0.0024 - val_accuracy: 0.9996 - val_loss: 5.0483e-04
Epoch 4/10
1014/1014 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9995 - loss: 9.4637e-04 - val_accuracy: 1.0000 - val_loss: 7.8426e-05
Epoch 5/10
1014/1014 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9999 - loss: 2.8816e-04 - val_accuracy: 0.9988 - val_loss: 0.0063
Epoch 6/10
1014/1014 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9988 - loss: 0.0030 - val_accuracy: 0.9996 - val_loss: 4.0011e-04
Epoch 7/10
1014/1014 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 1.0000 - loss: 1.4025e-04 - val_accuracy: 1.0000 - val_loss: 5.1549e-04
Epoch 8/10
1014/1014 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - ac

- Salvando os pesos finais da rede 
- Salvando o histórico de perda para cada iteração
- Salvando as saídas produzidas pela rede para cada um dos dados de teste

In [41]:
# PESOS FINAIS
model.save_weights('arquivos-brutos/pesos_finais.weights.h5')

# ERRO DE CADA ITERAÇÃO
perdas = historico.history['loss']

# Salvando o histórico de perda em um arquivo JSON
with open('arquivos-brutos/historico_perda.json', 'w') as f:
    json.dump(perdas, f)


# SAÍDAS PRODUZIDAS
# Fazendo inferência com o modelo treinado para obter as saídas
saidas = model.predict(x_train)

# Convertendo as saídas para uma lista para serialização
saidas_lista = saidas.tolist()

# Salvando as saídas em um arquivo JSON
with open('arquivos-brutos/saidas_teste.json', 'w') as f:
    json.dump(saidas_lista, f)

396/396 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


Testar o modelo

In [42]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}, Test loss: {test_loss}')

67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9998 - loss: 5.6769e-04
Test accuracy: 0.9990543723106384, Test loss: 0.00460610119625926
